# _Python for Scientific Data Analysis_

# SciPy

## Section 3: Statistics

_“There are three kinds of lies: Lies, Damned Lies, and Statistics”_

  Mark Twain (or was it Benjamin Disraeli?)

The ``scipy.stats`` subpackage contains different a large number of functions dedicated to the latter: frequentists statistics, probability, distributions, correlation, functions, different statistical tests, etc.   Want to generate a distribution with a known functional form?  this subpackage has it?  Is one distribution different than another?  How different? This subpackage has it. 

 In previous generations, a lot of these topics were covered in the venerable "Numerical Recipes" textbook (er, phonebook) for code in C and FORTRAN.   Your instructor and his advisor (and his advisor before him, most likely) learned numerical techniques from this source.   We are no longer in the dark ages of dial-up Internet and cellphones the size of books, but this text (written *before* such an era) IS the authority on this topic, in my opinion.  And the value of that text to highlight what is important even in Python remains.  
 
 Therefore, some of my discussion will basically look like a "_Numerical Recipes in SciPy_" (someone should write this textbook).
 
 The key topics we want to cover are as follows:
 
 * _Statistical Properties of Data_ - You have some data, how do you describe it?
 * _Distributions of Data_ - You have some data, how is the data-as-an-aggregate intelligible?  What patterns do it fit into?
 * _Statistical Tests of Data_ - You have some data, how sure are you that it differs from some other data, how sure are you that something you think is interesting is actually significant vs noise, etc.

 Similar to before, our key start-up commands are to import relevant libraries: numpy, matplotlib, and the stats subpackage of SciPy:
 

In [5]:
 
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import scipy

#and because this is a Jupyter notebook
%matplotlib inline
from matplotlib import rcParams
rcParams['figure.figsize']=[8,6]

### Properties of Data

We have some data.  How would you describe it?   Well, let's then start with some data:

In [6]:
A = np.array([[10, 14, 11, 7, 9.5], [8, 9, 17, 14.5, 12],
              [15, 7, 11.5, 10, 10.5], [11, 11, 9, 12, 14]])
              
print(A.shape) #this is a (4,5) array

(4, 5)


#### _Basic Properties_

Key properties of the data include the mean, median, standard deviation, and variance.  

* mean -- The mean of an array is $\Sigma_{i} A_{i} /N$, where $N$ is the number of elements of $A$.    Note, that if you call the program without any axis keyword, then the mean is calculated over the entire array.  Whereas the axis keyword specifies that the mean is calculated over a specific dimension.  E.g. ``mean=scipy.stats.tmean(A,axis=1)``.

* median -- the median of an array $x$ of length $n$ equals  $x_{(n+1)/2}$ if $n$ is odd and $0.5\times(x_{n/2} + x_{(n/2)+1} )$ if $n$ is even.   

* variance -- By "variance" here we usually mean the "average of the squared deviations from the mean": $S^{2}$ = $\Sigma_{i} (x_{i}-\bar{x})^{2}$/$N$, where $N$ equals the total population size.   This is also known as the _population variance_.     There is also the _sample variance_, where $S^{2}$ = $\Sigma_{i} (x_{i}-\bar{x})^{2}$/($n-1$) for $n$ samples.   

 The best way to think about the latter is that we are picking out a sample of $n$ from the larger population of $N$.   As such, we have an additional correction factor (the "Bessell correction", which is responsible for the $n-1$ in the denominator.   You can also think of this as a "degree of freedom": i.e. you estimate the variance from a random sample of $n$ independent measurements, so the dof equals $n$ minus the number of parameters estimated (i.e. the sample mean).

* standard deviation -- The standard deviation of a random variable, $\sigma$, is equal to the square root of its variance.  e.g. for a population standard deviation, 
 $\sigma$ = $\sqrt{\Sigma_{i} (x_{i}-\bar{x})^{2}/N}$.   Similarly, the sample standard deviation includes a $n-1$ factor in the denomintor, not $N$:
 $\sigma$ = $\sqrt{\Sigma_{i} (x_{i}-\bar{x})^{2}/(n-1)}$


We can call these from SciPy as follows:

* ``mean=scipy.stats.tmean(A,axis=axis)``
* ``median=scipy.ndimage.median(A)`` . I.e. Within SciPy, the function to compute the median is (confusingly) found in the ``ndimage`` subpackage, not stats.  
* ``variance = scipy.stats.tvar(A,ddof=ddof)``.
* ``standard_deviation = scipy.stats.tstd(A,ddof=ddof,axis=axis)``

Note here that the weirdness with the function calls.   The "mean" call is understandable (though why is it not in stats?) but median lives in another subpackage.  Similarly, the variance and standard deviation are actually computed from the _trimmed [variance/standard deviation]_ functions.  What this does is allow you to select a subset of the sample range in computing the variance or standard deviation, though of course if you do not put anything for this keyword, then the full range is considered.

For these and probably a bunch of other reasons, most often you want to use equivalent NumPy functions for these operations.   Here I list the functions with the major keywords and their default values.   So in NumPy ...

* ``mean=np.mean(A,axis=None,ddof=0)``
* ``median=np.median(A,axis=None)``
* ``variance=np.var(A,axis=None,ddof=0)``
* ``standard_deviation = np.std(A,axis=None,ddof=0)``

``axis=None`` says "compute ___ over entire array", whereas ``axis=[not None/some number]`` says compute over a specific axis.   Also, the ddof keyword refers to "delta degrees of freedom": the divisor in the calculations use $n-ddof$.  By default, ddof=0 (i.e. this is a maximum likelihood estimate of the variance for normally distributed variables).

Anyway, this all seems rather silly (who cares so long as you know which version you are using, right?).  Except that the default assumptions about what is _actually_ being computed varies from programming language to language and package to package.  

E.g. take the array ``[2. , 3. , 4. , 2.1, 2.2, 3.3]``

Here's what you get if you just ask NumPy what the mean, standard deviation and variance are without any keywords:

In [7]:
#NumPy

print('NumPy \n')
a=np.array([2. , 3. , 4. , 2.1, 2.2, 3.3])
print(np.mean(a))
#2.766666666666667

print(np.std(a))
#0.7318166133366716

print(np.var(a))
#0.5355555555555555


#and SciPy
print('')
print('Scipy \n')
#SciPy

print(scipy.stats.tmean(a))
#2.766666666666667

print(scipy.stats.tstd(a))
#0.8016649341630621   #uh oh!

print(scipy.stats.tvar(a))
#0.6426666666666666  #oh no!

NumPy 

2.766666666666667
0.7318166133366716
0.5355555555555555

Scipy 

2.766666666666667
0.8016649341630621
0.6426666666666666


Okay, maybe this is just some weirdness with SciPy vs NumPy ... nope!   Here are the equivalent commands in IDL

```
IDL> print,mean(a)
#2.76667

IDL> print,stdev(a)
#0.801665 #SciPy-like

IDL> print,variance(a)
#0.64266670 #SciPy-like again
```


The solution to reconcile these answers is to understand what exactly NumPy and SciPy are calculating.   E.g. NumPy's standard deviation and variance are population values (i.e. big "N" in the denominator), while SciPy's are sample standard deviations and variances (little "n" in the denominator).

In [8]:
#making NumPy's standard deviation agree with SciPy's
print(np.std(a,ddof=1))

#making SciPy's trimmed variance agree with NumPy's
print(scipy.stats.tvar(a,ddof=0))

0.8016649341630621
0.5355555555555555


So yeah, the NumPy implementations of these basic statistical properties are pretty user friendly ... but be very careful that you understand **exactly** what you are computing.

_Dealing with NaNs_

Now, if you have NaNs in your array these simple NumPy functions are probably going to crash when you try to compute the mean, median, variance or standard deviation.  Thankfully, NumPy has thought ahead and created functions that compute these statistical values _while ignoring_ NaNs:

``np.nanmean``, ``np.nanmedian``, ``np.nanvar``,``np.nanstd``.

#### _More Advanced Properties_

The above statistics can be done with either NumPy (recommended, provided you understand caveats) or SciPy (works but "Rube-Goldberg machine"-ish?).  For more advanced characteristics, SciPy -- in particular ``scipy.stats`` -- becomes more useful and is the preferred option.

Some more complicated properties include

* skew -- ``scipy.stats.skew`` -- Gives the relative weight to the right or left tail of a distribution.  For normally distributed data, the skewness should be about zero. For unimodal continuous distributions, a skewness value greater than zero means that there is more weight in the right tail of the distribution.  The sample skewness is computed as the Fisher-Pearson coefficient of skewness: $g_{1}$=$m_{3}$/$m_{2}^{1.5}$, where $m_{i} = \Sigma_{n=1}^{N} (x[n]-\bar{x})^{i}$/$N$

* kurtosis -- ``scipy.stats.kurtosis`` -- is the measure of the "tailed-ness" of a given distribution (i.e. how often outliers occur).   It equals the "fourth central moment" divided by the square of the variance (i.e. $\mu_{4}$/$S^{2}$= $\mu_{4}$/$\sigma^{4}$).   For a normal distribution the value of the kurtosis is 0.

* interquartile range -- ``scipy.stats.iqr``-- The interquartile range is the difference between the 75th and 25th percentile of the data.   It is an outlier-resistant measure of the dispersion of data.  

   Now this is great but what if you actually want the values for the 75th or 25th percentile?  I'm sure there is a scipy function to do this, but the simpler route is to just use NumPy.  E.g. ``np.nanpercentile([array],25)`` gives the 25th percentile value of an array.